# Axisym Punch-Through Shear Test

This example couples two domains via an zero-thickness interface. 

State - 2021-07-19 [RC]:

- Slide34 included with not verified material parameters. The push through test is running for a small step size
- Add the force control boundary conditions in analogy to single facet example
- Change of the parameters to brittle response leads to convergence problems at the transition to inelastic state - check the range which converges.
- Either initial or secant stiffness has been used     

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34_sw2 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization - Displacement-controlled

In [6]:
n_y_e = 10
n_inner_x_e = 5
n_outer_x_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[1:-1, -1],
    J=xd_upper.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)

In [7]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params =  dict(
     E_T=28000, gamma_T=500, K_T=0, S_T=0.005, c_T=6, bartau=3, 
     E_N=28000, S_N=0.005, c_N = 3, m = 0., f_t=2, f_c=100, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [8]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-0.5)
unloaded_inner = BCSlice(slice=xd_lower.mesh[0, :, 0, :],
                  var='u', dims=[0], value=0)
bc1 = [axisym, fixed_upper_support, loaded_lower]

In [9]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.01
s.tstep.fe_domain.serialized_subdomains

In [10]:
#bond_m.interact()

In [11]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.01(24), 
	 0.02(23), 
	 0.03(23), 
	 0.04(23), 
	 0.05(23), 
	 0.06(23), 
	 0.07(23), 
	 0.08

KeyboardInterrupt: 

# Plot the punch-through curve

In [52]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [53]:
F_to.shape

(34, 264)

In [54]:
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [55]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [57]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [58]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [59]:
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']

In [60]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [61]:
_, (ax, ax_omega) = plt.subplots(1,2, figsize=(10,5))
idx = -1
ax_u = ax.twinx()
ax.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax_omega.plot(x_m, omega_Nx[idx,:,:].flatten(), color='green', label='omega_N')
ax_omega.plot(x_m, omega_Tx[idx,:,:].flatten(), color='green', linestyle='dashed', label='omega_T')
ax.legend()
ax_omega.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-61-df7fb91e129e>:7: MatplotlibDeprecationWarning: Since 3.2, the parameter *where* must have the same size as [20.0 40.0 40.0 60.0 60.0 80.0 80.0 100.0 100.0 120.0 120.0 140.0 140.0
 160.0 160.0 180.0] in fill_betweenx(). This will become an error two minor releases later.
  ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)


In [62]:
sig_pi_Tx[-1]

array([[38.5644133 , 38.86304365],
       [38.86304365, 39.00972788],
       [39.00972788, 39.06623713],
       [39.06623713, 39.05299791],
       [39.05299791, 38.973491  ],
       [38.973491  , 38.821402  ],
       [38.821402  , 38.57339879],
       [38.57339879, 38.1653564 ]])

# FE discretization - Force-controlled

In [12]:
n_y_e = 10
n_inner_x_e = 5
n_outer_x_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[1:-1, -1],
    J=xd_upper.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)

In [13]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params = dict(
         E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
         E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
bond_m = Slide34(**material_params)

In [14]:
tf_force = TFBilinear()
tf_force.interact()

In [17]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower_slice= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0.5)
loaded_lower_dofs = loaded_lower_slice.dofs

load = 500
F = load / len(loaded_lower_dofs)

loaded_lower = [BCDof(var='f', dof=dof, value = F, time_function=tf_force  ) 
                     for dof in loaded_lower_dofs]

bc1 = [axisym, fixed_upper_support, loaded_lower]

In [18]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.02
s.tstep.fe_domain.serialized_subdomains

In [19]:
#bond_m.interact()

In [20]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00

TraitError: Each element of the 'bcond_list' trait of a BCondMngr instance must be an IBCond or None, but a value of [<ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6D45E0>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6C6D60>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6C6BD0>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6C6C20>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CEB96E00>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6D4590>] <class 'list'> was specified.